# Ferromagnetic Simulation

The following code provides an interactive simulation of an Ising model on a 2D lattice using the [Wolff algorithm](http://csml.northwestern.edu/resources/Reprints/lnp_color.pdf).

The following Julia libraries are required:

- Interact.jl
- Plots.jl
- Suppressor.jl

In [ ]:
using Interact, Plots, Suppressor

In [ ]:
N = 20
s_init = (rand(N,N) .> 0.5).*2 .- 1
print(size(s_init))

const Spin = Tuple{Integer,Integer}
const Bond = Set{Spin}

function neighboursof(loc::Spin)::Array{Spin,1}
	[(loc[1] < N ? [(loc[1]+1, loc[2])] : []);
		(loc[1] > 1 ? [(loc[1]-1, loc[2])] : []);
		(loc[2] < N ? [(loc[1], loc[2]+1)] : []);
		(loc[2] > 1 ? [(loc[1], loc[2]-1)] : [])]
end

"MCMC sampler for an Ising model on a 2D lattice using the Wolff algorithm,
generates the next sample given current state `s`, inverse temperature beta, and interaction strength J"
function sample(s, beta, J)
	sampledbonds = Set()
	spinstack = [(rand(1:N),rand(1:N))]
	cluster = Set()
	while !isempty(spinstack)
		spin = pop!(spinstack)
		push!(cluster, spin)
		for spin_ in neighboursof(spin)
			bond = Set((spin,spin_))
			bond in sampledbonds && continue
			s[spin[1],spin[2]] != sign(J)*s[spin_[1],spin_[2]] && continue
			p = 1 - exp(-2*beta*abs(J))
			rand() < p && push!(spinstack, spin_)
			push!(sampledbonds, bond)
		end
	end
	s_ = copy(s)
	for loc in cluster
		s_[loc[1],loc[2]] = -s_[loc[1],loc[2]]
	end
	s_
end

The simulation has two options: beta, which controls the inverse temperature of the simulation, and a toggle between ferromagnetic (J=1) and antiferromagnetic (J=-1) behaviour. Roughly speaking, at low beta values, the simulation will tend towards more disordered states, whereas at high beta values, it will behave in a more strictly ferromagnetic or antiferromagnetic way.

In [ ]:
fps = 3
timer = Observable(0.0)
@async while true
	sleep(1/fps)
	timer[] = timer[] + 1/fps
end

s = s_init
@manipulate for beta=0:0.02:1, mag=togglebuttons(["Ferromagnetic","Antiferromagnetic"]), t=timer
	global s
	J = mag == "Ferromagnetic" ? 1 : -1
	s = sample(s, beta, J)
	@suppress_err begin
		# Plots.jl gives a lot of GKS warnings here - probably a bug in the library
		heatmap(s, colorbar=false)
	end
end